<a href="https://colab.research.google.com/github/Lin6237/ai-finance-news-analysis/blob/main/AI_Finance_News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install requests
!pip install beautifulsoup4
!pip install matplotlib pandas wordcloud
!pip install yfinance yahoo_fin
!pip install transformers torch
!pip install lxml
!pip install smtplib
!pip install getpass
!pip install yfinance

ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib
ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


套件


In [31]:
import os
import smtplib
import getpass
import requests
import matplotlib.pyplot as plt
import pandas as pd
import time
import yfinance as yf
from yahoo_fin import stock_info
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from transformers import pipeline

AI 相關模型

In [32]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis")

Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


1. 爬取 Yahoo Finance 最新市場新聞

In [33]:
def get_yahoo_finance_news():
    """透過 Google News 爬取 Yahoo Finance 最新新聞"""
    google_news_url = "https://www.google.com/search?q=site:finance.yahoo.com+stock+market+news&tbm=nws"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(google_news_url, headers=headers)
    if response.status_code != 200:
        print("無法訪問 Google News，請檢查網路或網站是否封鎖爬蟲")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    news_data = []
    articles = soup.find_all("div", class_="BNeawe vvjwJb AP7Wnd")

    if not articles:
        print("未找到 Yahoo Finance 新聞，可能是 HTML 結構變更")
        return []

    for article in articles[:5]:  # 取前 5 則新聞
        try:
            title = article.get_text().strip()
            parent_div = article.find_parent("a")
            link = parent_div["href"] if parent_div else "無法取得網址"
            if link.startswith("/url?q="):
                link = link.split("&")[0].replace("/url?q=", "")

            # 爬取新聞內容
            content = "無法獲取新聞內容"
            if "yahoo.com" in link:
                article_response = requests.get(link, headers=headers)
                article_soup = BeautifulSoup(article_response.text, "html.parser")
                paragraphs = article_soup.find_all("p")
                content = " ".join([p.get_text() for p in paragraphs[:5]]) if paragraphs else "無法獲取新聞內容"
        except Exception as e:
            content = "新聞解析錯誤"

        news_data.append({"title": title, "url": link, "content": content})

    return news_data

2. AI 生成摘要與情緒分析

In [34]:
def process_news(news_list):
    """生成新聞摘要並進行情緒分析"""
    for news in news_list:
        content = news.get("content", "").strip()
        news["generated_summary"] = (
            summarizer(content, max_length=50, min_length=10, do_sample=False)[0]['summary_text']
            if content else "摘要不可用（原文缺失）"
        )
        news["sentiment"] = sentiment_analyzer(news["generated_summary"])[0]['label']

3. 視覺化數據：新聞情緒條形圖 & 詞雲

In [35]:
def visualize_results(news_list):
    """生成新聞情緒分析條形圖 & 詞雲，並儲存圖片"""
    sentiments = [news["sentiment"] for news in news_list]
    titles = " ".join([news["title"] for news in news_list])

    # 條形圖
    sentiment_counts = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
    for sentiment in sentiments:
        sentiment_counts[sentiment] += 1

    plt.figure(figsize=(6, 4))
    plt.bar(sentiment_counts.keys(), sentiment_counts.values(), color=["green", "red", "gray"])
    plt.xlabel("情緒分析結果")
    plt.ylabel("新聞數量")
    plt.title("Yahoo Finance 新聞情緒統計")
    plt.savefig("sentiment_chart.png")
    plt.close()

    # 詞雲
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(titles)
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("新聞標題關鍵字雲")
    plt.savefig("wordcloud.png")
    plt.close()

    print("✅ 圖表已儲存：sentiment_chart.png, wordcloud.png")

4. 爬取市場數據

In [36]:
def get_market_data():
    """使用 yfinance 獲取市場指數數據"""
    indices = {
        "S&P 500": "^GSPC",
        "Dow Jones": "^DJI",
        "Nasdaq": "^IXIC",
        "Apple": "AAPL",
        "Tesla": "TSLA"
    }

    market_data = {}
    for name, symbol in indices.items():
        try:
            ticker = yf.Ticker(symbol)
            price = ticker.history(period="1d")["Close"].iloc[-1]  # 取最新收盤價
            market_data[name] = f"${price:.2f}"
        except Exception as e:
            market_data[name] = "無法獲取數據"

    return market_data

5. 發送 Email，包含市場數據與分析結果

In [37]:
def send_email_with_market_data(news_list):
    """發送 Email，並附加市場指數、條形圖與詞雲"""

    sender_email = input("輸入你的 Gmail 地址: ")
    receiver_email = input("輸入收件人 Email: ")
    password = getpass.getpass("輸入你的 Gmail 應用程式密碼（不會顯示）: ")

    market_data = get_market_data()
    market_text = "\n".join([f"{key}: {value}" for key, value in market_data.items()])

    news_text = "\n".join([
        f"{n+1}. {news['title']}\n摘要: {news['generated_summary']}\n情緒: {news['sentiment']}\n網址: {news['url']}\n"
        for n, news in enumerate(news_list)
    ])

    subject = "AI 金融市場新聞與數據分析"
    body = f"📊 市場數據指數：\n{market_text}\n\n📰 AI 金融新聞分析：\n{news_text}"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    image_files = ["sentiment_chart.png", "wordcloud.png"]
    for image_file in image_files:
        with open(image_file, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={image_file}")
        msg.attach(part)

    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()
    print("📧 Email 發送成功，已附加市場數據與圖片！")

6. 主程式

In [38]:
if __name__ == "__main__":
    news_list = get_yahoo_finance_news()
    if news_list:
        process_news(news_list)
        display_results(news_list)
        visualize_results(news_list)
        send_email_with_market_data(news_list)
    else:
        print("⚠️ 無法獲取新聞，請稍後再試。")


📊 Yahoo Finance 新聞分析結果：

1. 📰 標題：Rocket to Buy Real Estate Broker Redfin for $1.75 Billion
   ✨ AI 摘要：Rocket Cos. (RKT) is pushing deeper into the property market with a deal to buy Redfin Corp. The deal values the real estate listing site at $1.75 billion.
   📌 情緒分析結果：POSITIVE
   🔗 網址：https://finance.yahoo.com/news/rocket-agrees-1-75-billion-105017116.html

2. 📰 標題：Ford to inject up to $4.76 billion into German business, FT reports
   ✨ AI 摘要：Ford will inject up to 4.4 billion euros ($4.76 billion) into its struggling German operations. U.S. carmaker is trying to revive its European business. Ford-Werke German arm will continue strategic transformation
   📌 情緒分析結果：POSITIVE
   🔗 網址：https://finance.yahoo.com/news/ford-inject-4-76-billion-092738554.html

3. 📰 標題：Wall Street Starts to Rethink Lofty S&P 500 Forecasts for 2025
   ✨ AI 摘要：For two consecutive years, stock-market prognosticators lifted their outlooks for the S&P 500 Index over and over again.
   📌 情緒分析結果：POSITIVE
   🔗 網址：http

<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 26032 (\N{CJK UNIFIED IDEOGRAPH-65B0}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 32862 (\N{CJK UNIFIED IDEOGRAPH-805E}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 25976 (\N{CJK UNIFIED IDEOGRAPH-6578}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 37327 (\N{CJK UNIFIED IDEOGRAPH-91CF}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 24773 (\N{CJK UNIFIED IDEOGRAPH-60C5}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-input-35-e221928131ca>:16: UserWarning: Glyph 32210 (\N{CJK UNIFIED IDEOGRAPH-7DD2}) missing from font(s) DejaVu Sans.
  plt.savefig("sentiment_chart.png")
<ipython-i

✅ 圖表已儲存：sentiment_chart.png, wordcloud.png
輸入你的 Gmail 地址: hinalin1017@gmail.com
輸入收件人 Email: hinalin1017@gmail.com
輸入你的 Gmail 應用程式密碼（不會顯示）: ··········
📧 Email 發送成功，已附加市場數據與圖片！
